In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
import gc

%matplotlib inline

In [2]:
plt.rc('figure',dpi=100);

In [3]:
avisos = pd.read_csv("../../Data/fiuba_entrenamiento/gian/avisos.csv")
postulaciones = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulaciones.csv")
postulantes = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulantes.csv")

postulantes['sexo']=postulantes['sexo'].astype('category')
postulantes['Doctorado']=postulantes['Doctorado'].astype('category')
postulantes['Master']=postulantes['Master'].astype('category')
postulantes['Otro']=postulantes['Otro'].astype('category')
postulantes['Posgrado']=postulantes['Posgrado'].astype('category')
postulantes['Secundario']=postulantes['Secundario'].astype('category')
postulantes['Terciario/Técnico']=postulantes['Terciario/Técnico'].astype('category')
postulantes['Universitario']=postulantes['Universitario'].astype('category')

avisos['nombre_zona']=avisos['nombre_zona'].astype('category')
avisos['tipo_de_trabajo']=avisos['tipo_de_trabajo'].astype('category')
avisos['nivel_laboral']=avisos['nivel_laboral'].astype('category')
avisos['nombre_area']=avisos['nombre_area'].astype('category')

postulaciones['sepostulo']=1
postulaciones.drop(['fechapostulacion','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)

In [4]:
vistas = pd.read_csv("../../Data/fiuba_entrenamiento/gian/vistas.csv")
vistas.head()

,idAviso,timestamp,idpostulante,dia,mes,semana,diadelasemana,hora
0,1111780242,2018-02-23 18:38:13.187,YjVJQ6Z,23,2,8,Viernes,18
1,1112263876,2018-02-23 18:38:14.296,BmVpYoR,23,2,8,Viernes,18
2,1112327963,2018-02-23 18:38:14.329,wVkBzZd,23,2,8,Viernes,18
3,1112318643,2018-02-23 18:38:17.921,OqmP9pv,23,2,8,Viernes,18
4,1111903673,2018-02-23 18:38:18.973,DrpbXDP,23,2,8,Viernes,18


In [5]:
vistas.drop(['timestamp','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)
vistas['cantidad']=0
vistas2=vistas.groupby(['idAviso','idpostulante'],as_index=False).agg({'cantidad':'count'})
vistas2=vistas2.rename(columns={'idAviso': 'idaviso'})
vistas2.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,2
1,48375,RwVdKR,1
2,169730,1KjXB,2
3,169730,2AKzxa,2
4,169730,6LJ64,1


In [6]:
vistas=vistas.groupby(['idpostulante'],as_index=False).agg({'cantidad':'count'});
vistas=vistas.rename(columns={'idAviso': 'idaviso'})
vistas.head()

,idpostulante,cantidad
0,0002q,15
1,0005E,58
2,000R8,14
3,001XE,26
4,003k9,32


In [7]:
vistas2=pd.merge(vistas2,vistas,on='idpostulante',how='left')

In [8]:
vistas2.head()

,idaviso,idpostulante,cantidad_x,cantidad_y
0,18,BolNL,2,28
1,48375,RwVdKR,1,15
2,169730,1KjXB,2,99
3,169730,2AKzxa,2,2
4,169730,6LJ64,1,23


In [9]:
vistas2['cantidad']=vistas2['cantidad_x']/vistas2['cantidad_y']
vistas2.drop(['cantidad_x','cantidad_y'],axis=1,inplace=True)
vistas=vistas2
vistas2.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,0.071429
1,48375,RwVdKR,0.066667
2,169730,1KjXB,0.020202
3,169730,2AKzxa,1.000000
4,169730,6LJ64,0.043478


In [10]:
postulaciones2= pd.read_csv("../../Data/fiuba_entrenamiento/gian/no-postulaciones/7.csv")
postulaciones2=postulaciones2.append(pd.read_csv("../../Data/fiuba_entrenamiento/gian/no-postulaciones/8.csv").sample(len(postulaciones2)//2).reset_index(drop=True))
print(len(postulaciones2))
postulaciones2

9906801


,idaviso,idpostulante,sepostulo
0,1112389495,5NbOjG,0
1,1112024109,1QP841b,0
2,1112490884,96XQ6N9,0
3,1111474259,qe2QJj1,0
4,1112211292,KBzbBJm,0
5,1112328485,3NzZv26,0
6,1112474766,akDjWwE,0
7,1112312385,aZpBJW,0
8,1112285628,akDWj55,0
9,1112322843,GNd0KNN,0


In [11]:
postulaciones.head()

,idaviso,idpostulante,sepostulo
0,1112257047,NM5M,1
1,1111920714,NM5M,1
2,1112346945,NM5M,1
3,1112345547,NM5M,1
4,1112237522,5awk,1


In [12]:
postulaciones=postulaciones.append(postulaciones2, ignore_index=True)
print(len(postulaciones))
postulaciones2=0
gc.collect()
postulaciones.drop_duplicates(['idaviso','idpostulante'],keep='first',inplace=True)
print(len(postulaciones))

16511335
16501929


In [13]:
postulaciones=pd.merge(postulaciones,postulantes,on='idpostulante')
postulaciones=pd.merge(postulaciones,avisos,on='idaviso')
postulaciones=pd.merge(postulaciones,vistas,on=['idaviso','idpostulante'],how='left')
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad
7183089,1112389353,5qvovM,1,FEM,30.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,NaN
5630019,1112427548,pzd6VKL,0,FEM,19.0,-,-,-,-,Graduado,...,True,True,False,False,False,False,False,False,False,NaN
12337620,1112377039,8MBz31D,0,FEM,49.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,NaN
998016,1111455368,BmZPKdz,1,MASC,25.0,-,-,-,-,-,...,False,False,False,False,False,True,False,False,False,0.142857
10507994,1112237261,axjr6m,0,FEM,36.0,-,-,-,Graduado,Graduado,...,False,True,False,False,False,False,False,False,False,NaN
13200127,1112415829,6rQ2mKM,0,FEM,29.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,NaN
12472782,1112291646,wVOmdRB,0,FEM,22.0,-,-,-,-,-,...,False,True,False,False,False,True,False,False,False,NaN
6224816,1112280937,pzeKaGL,1,FEM,25.0,-,-,-,-,Graduado,...,False,True,False,False,False,False,False,False,False,NaN
5648572,1112246707,akjl83m,0,MASC,200.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,NaN
8319608,1112295781,ekOBZ9p,0,FEM,34.0,-,-,Graduado,-,-,...,True,False,True,False,True,True,False,False,False,NaN


In [14]:
postulaciones.cantidad=postulaciones.cantidad.fillna(0)
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad
15781540,1112273550,LNKPwlj,0,FEM,22.0,-,-,-,-,Graduado,...,False,True,False,False,False,False,False,False,False,0.000000
2939764,1112296352,JBrm4dk,0,MASC,19.0,-,-,-,-,Graduado,...,True,False,False,False,False,False,False,False,False,0.000000
4746957,1112428730,dY5rLW0,0,MASC,23.0,-,-,-,-,Graduado,...,True,True,False,False,False,False,False,False,False,0.000000
3076334,1112368053,LNMPaJ5,0,FEM,25.0,-,-,-,-,Graduado,...,False,False,True,False,False,False,False,False,False,0.000000
10858591,1112263465,MVO8Kqe,1,FEM,22.0,-,-,-,-,Graduado,...,False,False,True,False,False,False,False,False,False,0.000000
11993022,1112286397,2zw5XpY,1,FEM,27.0,-,-,-,-,Graduado,...,False,False,True,False,False,False,False,False,False,0.000000
14348950,1112249378,EzllGw4,0,MASC,28.0,-,-,-,-,En Curso,...,False,True,False,False,False,False,False,False,False,0.000000
8126754,1112205105,8MamL3l,1,MASC,24.0,-,-,-,-,-,...,False,True,False,False,False,False,False,False,False,0.011111
11802224,1112389345,qe63WWk,1,FEM,26.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,0.000000
13940399,1112141104,6rQrJRR,0,NO_DECLARA,200.0,-,-,-,-,-,...,False,False,True,False,True,False,False,False,False,0.000000


In [15]:
train, test=train_test_split(postulaciones,test_size=0.05)
print("Train: ",len(train),"Test: ",len(test))
features=list(postulaciones.columns)
features.remove('idaviso')
features.remove('idpostulante')
features.remove('sepostulo')
features.remove('titulo')
features.remove('denominacion_empresa')

x_train=train[features]
y_train=train['sepostulo']

x_test=test[features]
y_test=test['sepostulo']
postulaciones=0
vistas2=0

Train:  15281800 Test:  804306


In [16]:
gc.collect()

156

In [17]:
c=lgb.LGBMClassifier(learning_rate=0.005,objective='binary',num_leaves=6000,max_depth=13,n_estimators=1200,colsample_bytree=0.8,n_jobs=-1,random_state=0,silent=False,subsample=0.8,sumsample_freq=0.5)

dt=c.fit(x_train,y_train)
#c = joblib.load("../../Data/fiuba_entrenamiento/gian/modelos/....pkl")

In [18]:
score=c.score(x_test,y_test)*100
print(score)

81.14710073031905


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [19]:
joblib.dump(c, "../../Data/fiuba_entrenamiento/gian/modelos/LGBMV15.pkl")

['../../Data/fiuba_entrenamiento/gian/modelos/LGBMV15.pkl']

In [20]:
prediccion = pd.read_csv("../../Data/fiuba_entrenamiento/test_final_100k.csv")
print(len(prediccion))
prediccion=pd.merge(prediccion,postulantes,on='idpostulante',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,avisos,on='idaviso',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,vistas,on=['idaviso','idpostulante'],how='left')
prediccion.cantidad=prediccion.cantidad.fillna(0)
prediccion.sample(10)

100000
100000
100000


,id,idaviso,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad
30224,87677,1112466900,Ym8YZV,MASC,33.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,0.008065
10541,87259,1112466797,86rzax,MASC,43.0,-,-,-,-,Graduado,...,False,True,True,True,False,False,False,False,False,0.000000
7476,89000,1112467020,jkdVJaO,FEM,25.0,-,-,-,-,-,...,False,False,True,False,False,False,False,True,False,0.030303
56573,49503,1112437977,ZvJLQP,FEM,33.0,-,-,-,-,-,...,False,True,False,False,False,False,False,False,False,0.019608
79752,27163,1112348675,eO2vEj,MASC,32.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.000000
4158,65295,1112458667,EedZ1o,MASC,30.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,0.006711
21669,54740,1112446957,QNrxrjq,MASC,22.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.037838
96087,95628,1112490233,1qlMR3,MASC,39.0,-,-,-,-,-,...,False,False,True,False,False,False,False,True,False,0.000000
4195,65087,1112458667,pzd5DXr,MASC,34.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.006452
29654,78777,1112464572,mzd9K23,NO_DECLARA,200.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,0.018265


In [21]:
x_final=prediccion[features]
y_final=c.predict_proba(x_final)
y_final

array([[0.82886698, 0.17113302],
       [0.81957543, 0.18042457],
       [0.8241493 , 0.1758507 ],
       ...,
       [0.9338592 , 0.0661408 ],
       [0.86908708, 0.13091292],
       [0.97704802, 0.02295198]])

In [22]:
y_final2=[]
print(c.classes_)
print(len(y_final))
for elem in y_final:
    y_final2.append(elem[1])
print(len(y_final2))

[0 1]
100000
100000


In [23]:
prediccion['sepostulo']=y_final2

In [24]:
prediccion.head()

,id,idaviso,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad,sepostulo
0,0,739260,6M9ZQR,FEM,42.0,Graduado,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,0.0,0.171133
1,1,739260,6v1xdL,MASC,30.0,-,-,Graduado,-,-,...,False,False,False,False,False,False,False,False,0.0,0.180425
2,2,739260,ezRKm9,FEM,36.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,0.0,0.175851
3,3,758580,1Q35ej,MASC,68.0,-,-,-,Graduado,Graduado,...,False,False,False,False,False,False,False,False,0.0,0.039668
4,4,758580,EAN4J6,FEM,32.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,0.0,0.037475


In [25]:
prediccion.drop(features,axis=1,inplace=True)
prediccion.drop(['idaviso','idpostulante','titulo','denominacion_empresa'],axis=1,inplace=True)
prediccion.head()

,id,sepostulo
0,0,0.171133
1,1,0.180425
2,2,0.175851
3,3,0.039668
4,4,0.037475


In [26]:
prediccion.to_csv("../../Data/fiuba_entrenamiento/gian/predicciones/prediccion30.csv",index=False)

In [27]:
prediccion.sepostulo.mean()

0.5276648882033403